# Pre-Proccesing

## To import all modules

In [1]:
!pip install seaborn
!pip install bertopic
from zhconv import convert
import pandas as pd
import re
import esupar
from supar.utils.transform import CoNLLSentence
from itertools import combinations
import os
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations
from umap import UMAP
from sklearn.cluster import KMeans


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


/Users/osakiyutaka/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## To make raw data tidy

### To load raw dataset

In [2]:
data = pd.read_csv('data/dataset.csv')

### To make tidy dataset

In [3]:
def make_tidy_data(df):
    """
    Preprocesses the given DataFrame by splitting the 'content' column into paragraphs,
    removing certain characters, exploding the DataFrame, adding a new column to indicate
    if a paragraph contains the character '仁', and removing any unnamed columns.

    Args:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The preprocessed DataFrame.
    """
    try:
        df['content'] = df['content'].apply(lambda x: x.split("\n\n"))
        df['content'] = df['content'].apply(lambda x: [re.sub(r'[「」]', '"', i) for i in x])
        df = df.explode('content')
        df['contain_ren'] = df['content'].apply(lambda x: 1 if '仁' in x else 0)
        df.reset_index(drop=True, inplace=True)
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    except:
        pass
    return df

In [4]:
df = make_tidy_data(data)

In [5]:
df.to_csv('data/modified_dataset.csv', index=False)

### To add time attributes

In [6]:
list_bookname = list(df["book"].unique())
list_book_time = [
    ['analects',-480, -350],
    ['mengzi',  -340, -250],
    ['liji',    -475, -221],
    ['xunzi',   -475, -221],
    ['xiao-jing',-475, -221],
    ['shuo-yuan',-206, 9],
    ['chun-qiu-fan-lu',-206, 9],
    ['han-shi-wai-zhuan',-180, -120],
    ['da-dai-li-ji',100, 200],
    ['bai-hu-tong',79, 92],
    ['xin-shu', -206, 9],
    ['xin-xu',-206, 9],
    ['yangzi-fayan',-33, 18],
    ['zhong-lun',25, 220],
    ['kongzi-jiayu',-206, 220],
    ['qian-fu-lun',102, 167],
    ['lunheng', 80, 80],
    ['taixuanjing',-33, 18],
    ['fengsutongyi',190, 200],
    ['kongcongzi',25, 265],
    ['shenjian',196, 220],
    ['zhong-jing',100,166],
    ['su-shu',-250,-186],
    ['xinyu',-196, -196],
    ['duduan',167, 258],
    ['caizhong-langji', 152, 192]
]
def map_year(year):
    return int(year/2 + 2000)
def set_time_attributes(df):
    """
    This function sets the time attributes "Start", "End", and "era" for each book in the input dataframe.

    Parameters:
    df (pandas.DataFrame): The input dataframe. It should contain a column named "book".

    Returns:
    pandas.DataFrame: The input dataframe with three new columns: "Start", "End", and "era". Each row represents a book.
    - "Start" is the start year of the book.
    - "End" is the end year of the book.
    - "era" is the average of the start year and the end year.
    """
    df["Start"] = None  # Initialize the "Start" column
    df["End"] = None  # Initialize the "End" column
    df["average_year"] = None
    df['mapped_year'] = None
    num = list_bookname.index(df.iloc[0]["book"])  # Get the index of the first book in the list of book names
    for i in range(df.shape[0]):  # For each row in the dataframe
        num = list_bookname.index(df.iloc[i]["book"])  # Get the index of the book in the list of book names
        df.loc[i, 'Start'] = list_book_time[num][1]  # Set the "Start" attribute
        df.loc[i, 'End'] = list_book_time[num][2]  # Set the "End" attribute
    for i in range(df.shape[0]):  # For each row in the dataframe
        df.loc[i, 'average_year'] = int((df.loc[i, 'Start']+df.loc[i, 'End'])/2)
    df['mapped_year'] = df['average_year'].apply(map_year)
    return df  # Return the dataframe with the new attributes

In [7]:
df = set_time_attributes(df)
df.head()

,book,chapter,content,contain_ren,Start,End,average_year,mapped_year
0,analects,學而,"子曰：""學而時習之，不亦說乎？有朋自遠方來，不亦樂乎？人不知而不慍，不亦君子乎？""",0,-480,-350,-415,1792
1,analects,學而,"有子曰：""其為人也孝弟，而好犯上者，鮮矣；不好犯上，而好作亂者，未之有也。君子務本，本立而道...",1,-480,-350,-415,1792
2,analects,學而,"子曰：""巧言令色，鮮矣仁！""",1,-480,-350,-415,1792
3,analects,學而,"曾子曰：""吾日三省吾身：為人謀而不忠乎？與朋友交而不信乎？傳不習乎？""",0,-480,-350,-415,1792
4,analects,學而,"子曰：""道千乘之國：敬事而信，節用而愛人，使民以時。""",0,-480,-350,-415,1792


## Information about dataset

In [8]:
print(f'The length of the DataFrame is {len(df)}', 
      f'The number of sentences that contain the character "仁" is {len(df[df["contain_ren"] == 1])}', sep='\n')

The length of the DataFrame is 12604
The number of sentences that contain the character "仁" is 921


In [9]:
#to see the number of chapter in three periods: ~202bc, 202bc-25ac, 25ac~
df_period1 = df[df['average_year'] <= -202]
df_period1_filtered = df_period1[['book', 'chapter']].drop_duplicates()
df_period2 = df[(df['average_year'] > -202) & (df['average_year'] <= 25)]
df_period2_filtered = df_period2[['book', 'chapter']].drop_duplicates()
df_period3 = df[df['average_year'] > 25]
df_period3_filtered = df_period3[['book', 'chapter']].drop_duplicates()
print("The number of chaprers in three periods:", len(df_period1_filtered)+len(df_period2_filtered)+len(df_period3_filtered))
print('The number of chapters in period 1:', len(df_period1_filtered))
print('The number of chapters in period 2:', len(df_period2_filtered))
print('The number of chapters in period 3:', len(df_period3_filtered))

The number of chaprers in three periods: 697
The number of chapters in period 1: 139
The number of chapters in period 2: 293
The number of chapters in period 3: 265


## To do word segmentation

### To load tidy dataset

In [10]:
df = pd.read_csv("data/modified_dataset.csv")

In [11]:
def word_segmentation(text):
    """
    This function segments the input text into words and returns the segmented words and their parts of speech.

    Parameters:
    text (str): The input text to be segmented.

    Returns:
    tuple: A tuple of two lists. The first list contains the segmented words. The second list contains the parts of speech of the segmented words.
    """
    conllu_data = nlp(text)
    word_segmented = conllu_data.values[1]
    word_segmented_pos = conllu_data.values[3]
    return word_segmented, word_segmented_pos

nlp = esupar.load("KoichiYasuoka/roberta-classical-chinese-base-upos")

def co_occurence_preparation(df, bookindex):
    """
    This function prepares a co-occurrence dataframe for a specific book in the input dataframe.

    Parameters:
    df (pandas.DataFrame): The input dataframe. It should contain the following columns: "book", "chapter", "sentences", "Start", and "End".
    bookindex (int): The index of the book to be processed.

    Returns:
    pandas.DataFrame: A dataframe with the following columns: "Item1", "Item2", "book", "chapter", "sentences_index", "contain_ren", "Start", and "End". Each row represents a pair of co-occurring words in a sentence of the specified book.
    """
    df_cooccurrence = pd.DataFrame(columns=['Item1', 'Item2', 'book', 'chapter', 'sentences_index', 'contain_ren', 'Start', 'End'])
    for index, row in df[df["book"] == df["book"].unique()[bookindex]].iterrows():
        print(df["book"][bookindex])
        try:
            word_segmented, word_segmented_pos = word_segmentation(row["sentences"])
            pairs = list(combinations(word_segmented, 2))
            tmp_df = pd.DataFrame(pairs, columns=['Item1', 'Item2'])
            tmp_df['book'] = row['book']
            tmp_df['chapter'] = row['chapter']
            tmp_df['sentences_index'] = index
            tmp_df['contain_ren'] = tmp_df.apply(lambda x: '仁' in x['Item1'] or '仁' in x['Item2'], axis=1)
            tmp_df['Start'] = row['Start']
            tmp_df['End'] = row['End']
            df_cooccurrence = pd.concat([df_cooccurrence, tmp_df], ignore_index=True)
        except:
            pass
    return df_cooccurrence

Some weights of RobertaModel were not initialized from the model checkpoint at KoichiYasuoka/roberta-classical-chinese-base-upos and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df_cooccurence_analect = co_occurence_preparation(df, len(df["book"].unique()))
df_ren = df_cooccurence_analect[df_cooccurence_analect["contain_ren"]==True]
df_ren = df_ren.reset_index(drop=True)
df_ren.to_csv('data/data_ren.csv', index=False)

## Word Embedding

In [ ]:
data = pd.read_csv('data/dataset.csv')

In [ ]:
nlp = esupar.load("KoichiYasuoka/roberta-classical-chinese-base-upos")

def word_segmentation(text):
    """
    This function performs word segmentation on the input text using a pre-trained model.

    Parameters:
    text (str): The input text to be segmented. If the input is not a string or is empty, the function returns an empty string.

    Returns:
    str: The segmented text, with each word separated by a space. If the input is not a string or is empty, the function returns an empty string.
    """
    if not text or not isinstance(text, str):
        return ''
    conllu_data = nlp(text)
    word_segmented = conllu_data.values[1]
    word_segmented = ' '.join(word_segmented)
    return word_segmented

Some weights of RobertaModel were not initialized from the model checkpoint at KoichiYasuoka/roberta-classical-chinese-base-upos and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Convert the content to Simplified Chinese
data['chapter'] = data['chapter'].apply(lambda x: convert(x, 'zh-cn'))
data['content'] = data['content'].apply(lambda x: convert(x, 'zh-cn'))

# Create 'book', 'chapter', 'sentences' structure
data = data[['book', 'chapter', 'content']]

data = set_time_attributes(data)


In [ ]:
data.head()

,book,chapter,content,Start,End,average_year,mapped_year
0,analects,学而,子曰：“学而时习之，不亦说乎？有朋自远方来，不亦乐乎？人不知而不愠，不亦君子乎？”\n\n有...,-480,-350,-415,1792
1,analects,为政,子曰：“为政以德，譬如北辰，居其所而众星共之。”\n\n子曰：“诗三百，一言以蔽之，曰‘思无...,-480,-350,-415,1792
2,analects,八佾,孔子谓季氏：“八佾舞于庭，是可忍也，孰不可忍也？”\n\n三家者以雍彻。子曰：“‘相维辟公，...,-480,-350,-415,1792
3,analects,里仁,子曰：“里仁为美。择不处仁，焉得知？”\n\n子曰：“不仁者不可以久处约，不可以长处乐。仁者...,-480,-350,-415,1792
4,analects,公冶长,子谓公冶长，“可妻也。虽在缧绁之中，非其罪也”。以其子妻之。\n\n子谓南容，“邦有道，不废...,-480,-350,-415,1792


Segmenting all data into words took much time, so thic code is an example using one chapter

In [ ]:
# Apply the 'word_segmentation' function to the 'content' column, write the results to a txt file, and save after each step
with open('data/word_segmentation_tmp.txt', 'w', encoding='utf-8') as f:
    for sentence in data['content']:
        seg_sentence = word_segmentation(sentence)
        f.write(seg_sentence + '\n')
        f.flush()  # Flush the file content to disk
        break

Token indices sequence length is longer than the specified maximum sequence length for this model (656 > 512). Running this sequence through the model will result in indexing errors


2024-06-04 16:06:04 INFO Loading the data


2024-06-04 16:06:04 INFO 
Dataset(n_sentences=1, n_batches=1, n_buckets=1)
2024-06-04 16:06:04 INFO Making predictions on the dataset


2024-06-04 16:06:25 INFO 0:00:20.238353s elapsed, 0.05 Sents/s


Loading pre-segmented txt file

In [ ]:
# read output.txt file, and load it into the data dataframe, as a new column called 'segmented'
if os.path.exists('data/word_segmentation.txt'):
    with open('data/word_segmentation.txt', 'r') as f:
        lines = f.readlines()
    data['segmented'] = lines
data.drop(columns=['content'], inplace=True)

In [ ]:
df = set_time_attributes(df)
# Add a new column 'contain_ren' to check if the sentence contains the character '仁'
data['contain_ren'] = data['segmented'].apply(lambda x: 1 if '仁' in x else 0)

In [ ]:
data.head()

,book,chapter,Start,End,average_year,mapped_year,segmented,contain_ren
0,analects,学而,-480,-350,-415,1792,子 曰 学 而 时 习 之 不 亦 说 乎 有 朋 自 远 方 来 不 亦 乐 乎 人 不 ...,1
1,analects,为政,-480,-350,-415,1792,子 曰 为 政 以 德 譬 如 北 辰 居 其 所 而 众 星 共 之 子 曰 诗 三百 一...,0
2,analects,八佾,-480,-350,-415,1792,孔子 谓 季 氏 八 佾 舞 于 庭 是 可 忍 也 孰 不 可 忍 也 三 家 者 以 雍...,1
3,analects,里仁,-480,-350,-415,1792,子 曰 里 仁 为 美 择 不 处 仁 焉 得 知 子 曰 不 仁 者 不 可 以 久 处 ...,1
4,analects,公冶长,-480,-350,-415,1792,子 谓 公冶 长 可 妻 也 虽 在 缧 绁 之 中 非 其 罪 也 以 其 子 妻 之 子...,1
